In [228]:
# import required libries
import os
import numpy as np
import spicy as sp
import pandas as pd

# define required path
raw_data = '../data/raw/'
curated_data = '../data/curated/'

In [ ]:
popu = pd.read_csv(raw_data + 'population.csv')

In [ ]:
required = ['SA2_code','2014', '2015', '2016', '2017', '2018', '2021']
popu = popu[required]

In [ ]:
# Select data in Victoria
popu = popu.loc[(popu['SA2_code'].astype(int)>199999999) & (popu['SA2_code'].astype(int)<300000000)]

In [ ]:
popu.to_csv(curated_data + 'population.csv')

In [229]:
suburb = pd.read_csv(curated_data + 'suburb.csv')
del suburb['Unnamed: 0']

In [230]:
# drop duplicate suburbs
suburb['Suburb'] = suburb['Suburb'].drop_duplicates()
suburb = suburb.dropna()

In [231]:
# read in dataset
properties = pd.read_csv(curated_data + 'properties.csv')

In [232]:
# calcuate the number of suburbs in properties
suburb_count = properties.groupby(['suburb'],as_index=False)['name'].count()
suburb_count.columns = ['suburb', 'count']

In [209]:
# prepare for joining
suburb_count['suburb'] = suburb_count['suburb'].str.upper()

In [211]:
# join the suburb and dataset with sa2
suburb_count = suburb[['Suburb', 'SA2_code']].join(suburb_count.set_index('suburb'), on = 'Suburb').dropna().reset_index(drop=True)

In [212]:
# calculate the number of properties in each sa2 area
sa2_count = suburb_count.groupby(['SA2_code'], as_index=False).sum()
sa2_count.columns = ['SA2_code', 'sum']

In [213]:
# merge the sum of sa2 properties data
suburb_count = suburb_count.join(sa2_count.set_index('SA2_code'), on = 'SA2_code')

In [215]:
# calculate the properties proportions of suburbs
suburb_count['popu_ratio'] = suburb_count['count'] / suburb_count['sum']

In [221]:
# merge the population dataframe
suburb_popu = suburb_count.join(popu.set_index('SA2_code'), on = 'SA2_code')

In [222]:
# calculate population of suburbs according to the properties ratio
years = ['2014', '2015', '2016', '2017', '2018', '2021']
suburb_popu[years] = suburb_popu[years].multiply(suburb_popu['popu_ratio'], axis="index").astype(int)

In [223]:
suburb_popu = suburb_popu.drop(['SA2_code', 'count', 'sum', 'popu_ratio'], axis=1)

In [224]:
suburb_popu.to_csv(curated_data + 'suburb_population.csv')

In [225]:
health = pd.read_csv(raw_data + 'health.csv')

In [226]:
health['Proportion'] = health['Proportion'].str.replace("*", "")

/tmp/ipykernel_614/1757025872.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  health['Proportion'] = health['Proportion'].str.replace("*","")


In [227]:
health.to_csv(curated_data + 'health.csv')